In [1]:
import math
from scipy.stats import norm
from scipy.optimize import fsolve

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [11]:
S0 = 56
K = 60
r = 0.0450
q = 0.015
T = 8/12
Vm = 6.95

In [12]:
def blackScholesVanilla(K , T , S , v , q , r , CP ):
    phi = 1 if CP == 'C' else -1
    dp = (1/(v*math.sqrt(T)))*(math.log(S/K) + ((r - q + (v**2)/2)*T) )
    dm = dp - (v*math.sqrt(T))
    vBS = phi*((norm.cdf(phi*dp)*(S*math.exp(-q*T))) - ((norm.cdf(phi*dm))*(K*math.exp(-r*T))))
    return vBS

In [13]:
def calculateIV(v , K , T , S , q , r , CP , Vm):
    
    modelV = blackScholesVanilla(K , T , S , v , q , r , CP )
    eq = modelV - Vm
    return eq

In [14]:
IV = fsolve(calculateIV , x0 = 0.25 , args =( K , T , S0 , q, r , 'P', Vm))

In [15]:
print(IV)

[0.29489447]


In [16]:
def trinomialTreePricer(S , K , v, r , q , T , N):
    deltaT = T/N
    u = math.exp(v*math.sqrt(3*deltaT))
    d = 1/u
    m = 1
    #f = math.exp((r-q)*(deltaT))
    df = math.exp(-r*deltaT)
    t1 = (r-q-((v**2)/2))
    t2 = math.sqrt(deltaT/(12*(v**2)))
    pU = (1/6) + (t1*t2) 
    pD = (1/6) - (t1*t2)
    pM = (2/3)
    V_N = []
    for i in range((2*N)+1):
        V_N.append(max(K - S*(u**(N-i)),0))
    
    for j in range(N-1,-1,-1):
        V = []
        for i in range((2*j) +1):
            p = df*((pU*V_N[i]) + (pM*V_N[i+1]) + (pD*(V_N[i+2])))
            V.append(p)
        V_N = V
    
    return V_N[0]

In [17]:
def trinomialTreeBSPricer(S , K , v, r , q , T , N):
    deltaT = T/N
    u = math.exp(v*math.sqrt(3*deltaT))
    d = 1/u
    m = 1
    #f = math.exp((r-q)*(deltaT))
    df = math.exp(-r*deltaT)
    t1 = (r-q-((v**2)/2))
    t2 = math.sqrt(deltaT/(12*(v**2)))
    pU = (1/6) + (t1*t2) 
    pD = (1/6) - (t1*t2)
    pM = (2/3)
    V_N = []
    for i in range((2*N)-1):
        V_N.append(blackScholesVanilla(K , deltaT , S*(u**(N-1-i)) , v , q , r , 'P' ))
    
    for j in range(N-2,-1,-1):
        V = []
        for i in range((2*j) +1):
            p = df*((pU*V_N[i]) + (pM*V_N[i+1]) + (pD*(V_N[i+2]))) 
            #American Pricer
            am = max(K - S*(u**(j-i)) , 0)
            p = max(p,am)
            V.append(p)
        V_N = V
    
    return V_N[0]

In [18]:
def trinomialTreeBSRPricer(S , K , v, r , q , T , N):
    t1 = trinomialTreeBSPricer(S , K , v, r , q , T , 2*N)
    t2 = trinomialTreeBSPricer(S , K , v, r , q , T , N)
    
    return (2*t1) - t2

In [ ]:

N = 10
tol = 1e-6
v = IV[0]
prevV = trinomialTreePricer(S0 , K , v , r , q , T , N)
N = 2*N
currV = trinomialTreePricer(S0 , K , v , r , q , T , N)

print(prevV ,currV)
while abs(currV - prevV) > tol:
    
    prevV = currV
    N = 2*N
    currV = trinomialTreePricer(S0 , K , v , r , q , T , N)
    
    print(N , prevV,currV)

print(N , currV)

7.016640631156409 6.960918418919231
40 6.960918418919231 6.925221645414716
80 6.925221645414716 6.958216486047864
160 6.958216486047864 6.945829994097656
320 6.945829994097656 6.946818861438186
640 6.946818861438186 6.94979380833941
1280 6.94979380833941 6.949427021215969
2560 6.949427021215969 6.9502054465658984
5120 6.9502054465658984 6.949952270128587
10240 6.949952270128587 6.950019597766911


In [22]:
N_fixed = N

In [23]:
#Secant Implementation
iv0 = 0.05
iv1 = 1
tol = 1e-4
while abs(iv0 - iv1) > tol:
    f1 = trinomialTreeBSRPricer(S0 , K , iv1 , r , q , T , N_fixed)
    f0 = trinomialTreeBSRPricer(S0 , K , iv0 , r , q , T , N_fixed)
    print(iv0 , f0 , iv1 , f1)
    iv2 = iv1 - (((f1-Vm)*(iv1-iv0))/(f1 - f0))
    iv0 = iv1
    iv1 = iv2
    
print(iv1 , trinomialTreeBSRPricer(S0 , K , iv1 , r , q , T , N_fixed))

0.05 2.0003063953777436 1 12.031901850030353
1 12.031901850030353 0.24694862430629816 3.799054802592438
0.24694862430629816 3.799054802592438 0.2726464377338102 4.077216854701962
0.2726464377338102 4.077216854701962 0.272903556733574 4.080008546023878
0.27290276963182886 4.079999999777234
